In [ ]:
import spacy
import pandas as pd
import csv
import time



In [ ]:
  copyrightpath = "/home/polymath/Desktop/GSOC2021/copyrightByMichael.csv"
  df = pd.read_csv(copyrightpath, error_bad_lines= False, delimiter="\t", quoting=csv.QUOTE_NONE)
  # data.dropna(inplace=True)
  data = df.iloc[:10000,:]
  data['Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f'] = data['Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f'].astype(str)
  data['Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f'] = data['Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f'].str.lower()
  nlp = spacy.load("en_core_web_sm")


In [ ]:
def entityCheck(listA, listB, index, hit_index):
    THE_PROBABLE_LOGIC_POS_CHECK = ['NOUN', 'NUM', 'PROPN', 'PROPN']
    THE_PROBABLE_LOGIC_NER_CHECK = ['DATE', 'PERSON', 'CARDINAL', 'ORG']
    if set(THE_PROBABLE_LOGIC_POS_CHECK).intersection(set(listA["POS_TAG"])):
        if set(THE_PROBABLE_LOGIC_NER_CHECK).intersection(set(listB["Values"])):
                for _values in listB["Entity"]:
                    try:
                        date_object = datetime.datetime.strptime(_values, '%Y').year
                        if date_object:
                            print("It is a hittttt")
                            hit_index.append(index)
                            break
                    except (TypeError, ValueError) as e:
                        pass
                    # if THE_PROBABLE_LOGIC_NER_CHECK[0] in _values:
                    #     print("It is a hit!!!")
                    #     try:
                    #         hit_index.append(index)
                    #     except:
                    #         print(index)
        else:
            print("It is a miss!!!")
    return hit_index

In [ ]:
start = time.time()
final_indexes = []
temp_var = []
hit_index = []
for index, row in data.iterrows():
    text = data.loc[index, 'Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f']
    text_split = text.split(",")
    data.loc[index,'new_tag'] = str(text_split[-1])

    doc = nlp(text)

    if type(text) == float:
        continue
    
    ## Lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)

    
    # Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)

    
    # Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)

    ## List joining
    list_of_copyrights = " ".join(map(str,filtered_sentence))
    substring = "( c )"
    cp_symbol = '\xa9'
    
    if "copyright" not in list_of_copyrights:
        if substring in list_of_copyrights:
            list_of_copyrights = list_of_copyrights.replace(substring, "copyright")
        
        elif cp_symbol in list_of_copyrights:
            list_of_copyrights = list_of_copyrights.replace(cp_symbol, "copyright")
   
    ## Implementing NER and POS Tags after normalization
    doc2 = nlp(list_of_copyrights)

    ent_dict = {}

    full_table_ner = { "Entity": [], "Values": []}

    for x in doc2.ents:
        ent_dict[x.text] = x.label_

    for key in ent_dict:
        full_table_ner["Entity"].append(key)
        full_table_ner["Values"].append(ent_dict[key])

    pos_dict = {}
    full_table_pos = { "Entity": [], "POS_TAG": []}

    for token in doc:
        if not token.is_punct | token.is_space:
            pos_dict[token.text] = token.pos_

    for key in pos_dict:
        full_table_pos["Entity"].append(key)
        full_table_pos["POS_TAG"].append(pos_dict[key])

    entityCheck(full_table_pos, full_table_ner, index, hit_index)


print(hit_index)
expected_time = (time.time() - start)
print(expected_time)

In [ ]:
for i in hit_index:
    data.loc[i,'Hit&Miss'] = "t"

data["Hit&Miss"].fillna("f", inplace=True)
data.to_csv('splitted_copyrightsAlgorifyNewtag.csv', index=False)

In [ ]:
data.head()

In [ ]:
counter = 0
total_counter = 0
for index, row in data.iterrows():
    original_tag = data.loc[index, 'new_tag']
    algorithm_tag = data.loc[index,'Hit&Miss']

    if original_tag == "t":
        total_counter += 1
        if algorithm_tag == original_tag:
            counter += 1
    
accuracy_score = (counter/total_counter) * 100

In [ ]:
print(accuracy_score)

In [ ]:
all_false = data["Hit&Miss"] == "t"
all_false_df = data[all_false]
all_false_df.to_csv('splitted_copyrightsAlgorifysubDF_TRUEE.csv', index=False)

In [ ]:
all_true = data["Hit&Miss"] == "f"
all_true_df = data[all_true]
all_true_df.to_csv('splitted_copyrightsAlgorifysubDF_FALSE.csv', index=False)

In [ ]:
import datetime

list_date = ["copyright", "1998", "1999", "1997", "1", "hfjhf"]

for val in list_date:
    try:
        b = datetime.datetime.strptime(val, '%Y').year
        if b:
            print(b)
    except (TypeError, ValueError) as e:
        pass